In [ ]:
%load_ext autoreload
%autoreload 2

# Create and use the app

1. Create the app
1. Pick the languages
1. Load the NLP models
1. Load the books


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from transformers.pipelines import pipeline

from interleave_epub.epub.epub import EPub
from interleave_epub.interleave.constants import (
    hug_model_name_tmpl,
    hug_trad_cache_fol,
    sent_model_names,
    spa_model_cache_fol,
    spa_model_names,
)
from interleave_epub.interleave.interactive import InterleaverInteractive
from interleave_epub.nlp.cached_pipe import TranslationPipelineCache
from interleave_epub.nlp.local_spacy_model import spacy_load_local_model


## Constants


### Ep paths


In [ ]:
epub_folder_path = Path("~").expanduser() / "snippet" / "datasets" / "ebook"

epub_paths = {
    "fr": epub_folder_path / "Gaston_Leroux_-_Le_Mystere_de_la_chambre_jaune.epub",
    "en": epub_folder_path / "mystery_yellow_room.epub",
}

### Language tags


In [ ]:
lts = ["fr", "en"]


## App


In [ ]:
ii = InterleaverInteractive()


In [ ]:
# set the lang tags
ii.set_lang_tag("fr", "src")
ii.set_lang_tag("en", "dst")


In [ ]:
# load the models
ii.load_nlp()


In [ ]:
# load the books
ii.add_book(epub_paths["fr"], "src", "Chambre Jaune 2")
ii.add_book(epub_paths["en"], "dst", "Yellow Room")


In [ ]:
an_epub = ii.epubs["src"]
a_chap = an_epub.chapters[0]
a_par = a_chap.paragraphs[0]
a_par


In [ ]:
ii.change_chapter_curr("forward")


In [ ]:
ii.align_auto()


In [ ]:
# ii.aligner.compute_similarity()


In [ ]:
al = ii.aligners[ii.ch_id_pair_str]


In [ ]:
fig, ax = plt.subplots()
ax.imshow(al.sim.T[::-1, :])


In [ ]:
# ii.aligner.align_auto(min_sent_len=5)


In [ ]:
fig, ax = plt.subplots()
ax.scatter(al.all_good_ids_src, al.all_good_ids_dst_max, s=0.1)
ax.plot([0, al.sent_num_src], [0, al.sent_num_dst], linewidth=0.3)
fit_y = al.fit_func([0, al.sent_num_src])
ax.plot([0, al.sent_num_src], fit_y)
# ax.plot(all_good_i_rescaled, all_good_max_rescaled, linewidth=0.9)
ax.plot(al.all_ids_src, al.all_ids_dst_max, linewidth=0.9)
# ax.set_title(f"Matching")


In [ ]:
fig, ax = plt.subplots(figsize=(14, 19))

# # plot the fancy ones
# ax.plot(
#     al.all_ids_src,
#     al.all_ids_dst_max,
#     marker="x",
#     color="r",
#     linestyle="",
#     alpha=0.9,
# )

# # plot the greedy ones, where they are different
# bad_ids_src = []
# bad_ids_dst = []
# for i in range(len(al.all_good_ids_src)):
#     if al.all_ids_dst_max[i] != al.all_good_ids_dst_max[i]:
#         bad_ids_src.append(al.all_good_ids_src[i])
#         bad_ids_dst.append(al.all_good_ids_dst_max[i])
# ax.scatter(
#     bad_ids_src,
#     bad_ids_dst,
#     marker="o",
#     color="r",
#     alpha=0.5,
# )

ax.scatter(
    al.all_good_ids_src_rescaled,
    al.all_good_ids_dst_max_rescaled,
    marker="x",
    color="r",
    s=10,
    alpha=0.8,
)

sid = 0
for par in al.ch_src.paragraphs:
    num_sents = len(par.sents["orig"])
    sid += num_sents
    ax.axvline(sid, c="k")
sid = 0
for par in al.ch_dst.paragraphs:
    num_sents = len(par.sents["orig"])
    sid += num_sents
    ax.axhline(sid, c="k")

ax.imshow(al.sim.T)


In [ ]:
# ii.aligner.compute_ooo_ids()


In [ ]:
(
    al.curr_id_src,
    al.curr_id_dst_interpolate,
    al.all_ids_dst_max[al.curr_id_src],
)


Almost done:

1. Manually set the dst id
1. Add curr src id to the fixed src ids
1. Recompute ooo ids, skipping fixed
1. Find the first src id to fix
1. Get the best guess for dst
1. Repeat


In [ ]:
id_dst_correct = 5
ii.pick_dst_sent(id_dst_correct)

In [ ]:
(
    al.curr_id_src,
    al.curr_id_dst_interpolate,
    al.all_ids_dst_max[al.curr_id_src],
)


In [ ]:
good_src_to_dst = {
    cs_src: cs_dst
    for cs_src, cs_dst in zip(
        al.all_good_ids_src_rescaled,
        al.all_good_ids_dst_max_rescaled,
    )
}


In [ ]:
from collections import Counter
from itertools import groupby

th_consensus = 0.6

# pair up sentences id in the chapter and the paragraph id they belong to
cs_p_src_ids = [
    {"cs_src_id": cs_src_id, "par_src_id": al.ch_src.cs_to_ps[cs_src_id][0]}
    for cs_src_id in al.all_good_ids_src_rescaled
]

# group the pairs on the paragraphs
# par_src_id: the src paragraph
# cs_p_src_par_ids: the src_chap_sent_id, src_par_id pairs for this src_paragraph
for par_src_id, cs_p_src_par_ids in groupby(cs_p_src_ids, lambda x: x["par_src_id"]):
    # the src chapter sentence ids for this src paragraph
    cs_src_ids = list(cs_p_src["cs_src_id"] for cs_p_src in cs_p_src_par_ids)
    num_sents_src = len(cs_src_ids)
    print(par_src_id, cs_src_ids, num_sents_src)

    # search for the cs_p_src id in the good_src_rescaled dict
    # if you find it, get the corresponding dst cs_src_id
    # -> we are iterating over the all_good_ids_src_rescaled sooo
    # extract all the paragraphs those sentences belong to
    par_dst_ids = []
    for cs_src_id in cs_src_ids:
        if cs_src_id in good_src_to_dst:
            # the chapter_sentence_dst id
            cs_dst_id = good_src_to_dst[cs_src_id]
            # dst (paragraph, ps) id
            ps_dst_id = al.ch_dst.cs_to_ps[cs_dst_id]
            # get only the paragraph
            par_dst_ids.append(ps_dst_id[0])
        else:
            print(f"Very unexpected, missing {cs_src_id}.")

    # if we have no good sentences we would have no group in the groupy
    if len(par_dst_ids) == 0:
        print(f"Very unexpected, len=0 for {par_src_id}.")
        continue

    # decide if there is a consensus on the paragraphs
    par_dst_ids_count = Counter(par_dst_ids)
    print(f"\t{par_dst_ids_count}")
    par_dst_mc = par_dst_ids_count.most_common()[0]
    par_dst_mc_id = par_dst_mc[0]
    par_dst_mc_count = par_dst_mc[1]

    if par_dst_mc_count / num_sents_src > th_consensus:
        print(f"\tMatching par {par_src_id} {par_dst_mc_id}")
    else:
        print(f"\tNo matching. ------")

    # if len(par_dst_ids) > 3: break


In [ ]:
par_dst_ids_count

In [ ]:
par_dst_ids_count.most_common()

In [ ]:
par_dst_ids_count.most_common()[0][1]
